In [3]:
import seaborn

%load_ext autoreload
%pylab
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
from pylab import rcParams
rcParams['figure.figsize'] = 11, 4
seaborn.set_style("whitegrid")
seaborn.despine()

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
from mppfnet.mp_network import MPNetwork
from mppfnet.mp_problem import MPProblem
import gridopt
import pfnet

In [5]:
mp = MPNetwork()
mp.load('./data/case32.art')

In [6]:
mp.set_prices()

# bus voltage angles
mp.set_flags(pfnet.OBJ_BUS, pfnet.FLAG_VARS, pfnet.BUS_PROP_NOT_SLACK, pfnet.BUS_VAR_VANG)

# slack gens active powers
mp.set_flags(pfnet.OBJ_GEN, pfnet.FLAG_VARS, pfnet.GEN_PROP_SLACK,pfnet.GEN_VAR_P)

# gens active powers
mp.set_flags(pfnet.OBJ_GEN,pfnet.FLAG_BOUNDED,pfnet.GEN_PROP_ANY,pfnet.GEN_VAR_P)

p = MPProblem(mp)
p.add_function(pfnet.FUNC_TYPE_NETCON_COST, 1.0)
p.add_constraint(pfnet.CONSTR_TYPE_LBOUND)  # generator bounds
p.add_constraint(pfnet.CONSTR_TYPE_DCPF)  # power flow
p.analyze()
x = p.get_init_point()
p.eval(x)
p.construct_problem()

In [7]:
import numpy as np
from optalg.opt_solver import OptSolverIQP, QuadProblem, OptSolverLCCP
from scipy.sparse import triu, bmat
Hx = p.Hphi + p.Hphi.T - triu(p.Hphi)
gx = p.gphi - Hx*x

g = gx / mp.base_power
H = Hx / mp.base_power
A = p.A
b = p.b
l = p.l
u = p.u

problem = QuadProblem(H, g, A, b, l, u)
solver = OptSolverIQP()
solver.set_parameters({'quiet': False, 'tol': 1e-7})
problem.show()

In [ ]:
H.shape

In [ ]:
solver.solve(problem)
print(solver.get_status())
problem.x

In [ ]:
mp.generate_solar_profiles()
plot(mp.solar_profile_map[0])

In [ ]:
mp.generate_load_profiles()

In [ ]:
mp.get_load(1, 1).P

In [ ]:
plot(mp.load_profile_map[0])